In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.layers import Dense
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [2]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful']

In [3]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

def load_data(test_size):
    x,y = [],[]
    for file in glob.glob("../input/ravdess-emotional-song-audio/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc = True, chroma = True, mel = True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size = test_size, random_state = 0) 

x_train, x_test, y_train, y_test = load_data(test_size = 0.2) 

In [4]:
set(y_train)

{'calm', 'fearful', 'happy'}

In [5]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

In [6]:
x_traincnn = np.expand_dims(x_train, axis=2)
x_testcnn = np.expand_dims(x_test, axis=2)

In [7]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [8]:
model = Sequential()

model.add(Conv1D(filters = 32, kernel_size = 5, activation = 'relu', input_shape = [180, 1]))
model.add(BatchNormalization())
model.add(MaxPool1D(pool_size = 8, strides = 4))
model.add(Dropout(0.25))

model.add(Conv1D(filters = 32, kernel_size = 5, activation = 'relu')) 
model.add(MaxPool1D(pool_size = 8, strides = 4))
model.add(Dropout(0.25))

model.add(Conv1D(filters = 64, kernel_size = 3, activation = 'relu')) 
model.add(MaxPool1D(pool_size = 8, strides = 3, padding = "same"))
model.add(Dropout(0.25))

model.add(Conv1D(filters = 128, kernel_size = 2, activation = 'relu', padding = "same")) 
model.add(MaxPool1D(pool_size = 8, strides = 4, padding = "same"))

model.add(Conv1D(filters = 256, kernel_size = 3, activation = 'relu', padding = "same")) 
model.add(MaxPool1D(pool_size = 8, strides = 2, padding = "same"))


model.add(Flatten())

model.add(Dense(100, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))  

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 176, 32)           192       
_________________________________________________________________
batch_normalization (BatchNo (None, 176, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 43, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 43, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39, 32)            5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 32)             0

In [10]:
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
cnnhistory = model.fit(x_traincnn, y_train, batch_size = 16, epochs = 700, validation_data = (x_testcnn, y_test)) 

Epoch 1/700
28/28 [==============================] - 0s 17ms/step - loss: 1.1117 - accuracy: 0.3537 - val_loss: 1.0822 - val_accuracy: 0.3514
Epoch 2/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0749 - accuracy: 0.4059 - val_loss: 0.9960 - val_accuracy: 0.5045
Epoch 3/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0637 - accuracy: 0.4218 - val_loss: 1.0048 - val_accuracy: 0.5856
Epoch 4/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0114 - accuracy: 0.4558 - val_loss: 0.9789 - val_accuracy: 0.5135
Epoch 5/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0079 - accuracy: 0.4853 - val_loss: 0.9283 - val_accuracy: 0.5315
Epoch 6/700
28/28 [==============================] - 0s 6ms/step - loss: 0.9964 - accuracy: 0.4512 - val_loss: 1.0077 - val_accuracy: 0.4685
Epoch 7/700
28/28 [==============================] - 0s 6ms/step - loss: 0.9892 - accuracy: 0.4853 - val_loss: 0.9355 - val_accuracy: 0.5586
Epoch 8/700


In [12]:
from keras.callbacks import History 
print("Accuracy = ", cnnhistory.history['val_accuracy'][-1])

Accuracy =  0.9189189076423645
